# Moisture/Temperature-based radiative heating

## Import Package

In [1]:
import h5py
import numpy as np
import xarray as xr

from tqdm import tqdm
from matplotlib import pyplot as plt

## Load data

In [2]:
# Load linear response functions
with h5py.File("/home/b11209013/2025_Research/Obs/Files/RRTMG_LRF.h5", "r") as f:
    q_lw = np.array(f.get("q_lw"))[::-1, ::-1]
    t_lw = np.array(f.get("t_lw"))[::-1, ::-1]
    q_sw = np.array(f.get("q_sw"))[::-1, ::-1]
    t_sw = np.array(f.get("t_sw"))[::-1, ::-1]

# Load ERA5 data
## Load reference profile
with h5py.File("/home/b11209013/2025_Research/Obs/Files/era5_qmean.h5", "r") as f:
    lev = np.array(f.get("lev"))
    q_mean = np.array(f.get("q"))*1000.0 # convert to g/kg

with h5py.File("/home/b11209013/2025_Research/Obs/Files/era5_tmean.h5", "r") as f:
    t_mean = np.array(f.get("t"))

## Load tropical profiles
with xr.open_dataset("/work/b11209013/2024_Research/ERA5/q/q_sub.nc", chunks={}) as f:
    q_tropics = f["q"].values*1000.0 # convert to g/kg

with xr.open_dataset("/work/b11209013/2024_Research/ERA5/t/t_sub.nc", chunks={}) as f:
    t_tropics = f["t"].values

# Calculate perturbations
q_perturb = q_tropics - q_mean[None, :, None]
t_perturb = t_tropics - t_mean[None, :, None]

nt, nz, nx = q_perturb.shape

## Compute radiation from moisture and temperature

In [3]:
# reshape data
q_reshape = q_perturb.transpose(1,0,2).reshape(nz, nt*nx)
t_reshape = t_perturb.transpose(1,0,2).reshape(nz, nt*nx)

# Calculate radiative perturbations
q_lw_rad_perturb = q_lw @ q_reshape
q_sw_rad_perturb = q_sw @ q_reshape
t_lw_rad_perturb = t_lw @ t_reshape
t_sw_rad_perturb = t_sw @ t_reshape

# Reshape back to (time, lev, lon)
q_lw_rad_perturb_3d = q_lw_rad_perturb.reshape(-1, nt, nx).transpose(1,0,2)
q_sw_rad_perturb_3d = q_sw_rad_perturb.reshape(-1, nt, nx).transpose(1,0,2)
t_lw_rad_perturb_3d = t_lw_rad_perturb.reshape(-1, nt, nx).transpose(1,0,2)
t_sw_rad_perturb_3d = t_sw_rad_perturb.reshape(-1, nt, nx).transpose(1,0,2)

## Save to file

In [4]:
with h5py.File("/work/b11209013/2025_Research/moisture_rad.h5", "w") as f:
    f.create_dataset("q_lw", data=q_lw_rad_perturb_3d)
    f.create_dataset("q_sw", data=q_sw_rad_perturb_3d)
    f.create_dataset("t_lw", data=t_lw_rad_perturb_3d)
    f.create_dataset("t_sw", data=t_sw_rad_perturb_3d)